# Visualize Candles with Open Interest

This notebook fetches candle and open interest data, then creates a combined visualization.

In [1]:
import asyncio
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from core.data_sources.clob import CLOBDataSource
import pandas as pd

/opt/anaconda3/envs/quants-lab/lib/python3.12/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


## Configuration

In [2]:
# Configuration parameters
connector_name = "binance_perpetual"
trading_pair = "BTC-USDT"
interval = "1h"
days = 7

## Fetch Data

In [3]:
# Initialize data source
clob = CLOBDataSource()

# Load any existing caches
clob.load_candles_cache()
clob.load_oi_cache()

## Data Inspection

In [9]:
candles = await clob.get_candles_last_days(connector_name, trading_pair, interval, days)
# Display candle data summary
print("Candles DataFrame:")
print(candles.data.head())
print(f"\nShape: {candles.data.shape}")
print(f"Date range: {candles.data.index.min()} to {candles.data.index.max()}")

Candles DataFrame:
                     timestamp     open     high      low    close    volume  \
timestamp                                                                      
2025-09-25 16:00:00 1758816000 111538.4 111636.6   110852 110907.9  5431.493   
2025-09-25 17:00:00 1758819600   110908   110930 108578.2 108793.9 26570.236   
2025-09-25 18:00:00 1758823200 108793.9 109759.7 108574.1 109708.3 12498.301   
2025-09-25 19:00:00 1758826800 109708.4 109926.7   109250 109320.5  6038.849   
2025-09-25 20:00:00 1758830400 109320.5 109447.9   109005 109126.8  3823.724   

                     quote_asset_volume  n_trades  taker_buy_base_volume  \
timestamp                                                                  
2025-09-25 16:00:00  604182109.17840004    159025               2563.783   
2025-09-25 17:00:00 2915653481.23479986    467363              11183.231   
2025-09-25 18:00:00  1364142489.3032999    292958               6412.743   
2025-09-25 19:00:00  661695061.64390004 

In [10]:
oi_data = clob._oi_cache[(connector_name, trading_pair, interval)]

# Display OI data summary
if not oi_data.empty:
    print("\n=== Open Interest Data Summary ===")
    print(f"Total records: {len(oi_data)}")
    print(f"Date range: {oi_data.index.min()} to {oi_data.index.max()}")
    print(f"\nColumns available: {list(oi_data.columns)}")
    
    if 'sum_open_interest' in oi_data.columns:
        print(f"\nOpen Interest Stats:")
        print(f"  Current: {oi_data['sum_open_interest'].iloc[-1]:,.2f}")
        print(f"  Average: {oi_data['sum_open_interest'].mean():,.2f}")
        print(f"  Max: {oi_data['sum_open_interest'].max():,.2f}")
        print(f"  Min: {oi_data['sum_open_interest'].min():,.2f}")
    
    if 'sum_open_interest_value' in oi_data.columns:
        print(f"\nOI Value Stats (USD):")
        print(f"  Current: ${oi_data['sum_open_interest_value'].iloc[-1]:,.0f}")
        print(f"  Average: ${oi_data['sum_open_interest_value'].mean():,.0f}")
        print(f"  Max: ${oi_data['sum_open_interest_value'].max():,.0f}")
        print(f"  Min: ${oi_data['sum_open_interest_value'].min():,.0f}")
    
    print(f"\nFirst 5 records:")
    display(oi_data.head())
    
    print(f"\nLast 5 records:")
    display(oi_data.tail())
else:
    print("No OI data available")


=== Open Interest Data Summary ===
Total records: 168
Date range: 2025-09-25 15:00:00 to 2025-10-02 14:00:00

Columns available: ['sum_open_interest', 'sum_open_interest_value', 'cmc_circulating_supply', 'trading_pair']

Open Interest Stats:
  Current: 93,393.43
  Average: 87,416.20
  Max: 93,393.43
  Min: 84,552.98

OI Value Stats (USD):
  Current: $11,166,908,639
  Average: $9,828,007,906
  Max: $11,166,908,639
  Min: $9,226,915,939

First 5 records:


,sum_open_interest,sum_open_interest_value,cmc_circulating_supply,trading_pair
timestamp,,,,
2025-09-25 15:00:00,85906.973,9579704547.55116463,19925746,BTC-USDT
2025-09-25 16:00:00,86039.23,9597333469.60668564,19925746,BTC-USDT
2025-09-25 17:00:00,86062.448,9546210250.81119919,19925746,BTC-USDT
2025-09-25 18:00:00,84808.785,9226915939.0095005,19925746,BTC-USDT
2025-09-25 19:00:00,85487.441,9378690372.20439911,19925746,BTC-USDT



Last 5 records:


,sum_open_interest,sum_open_interest_value,cmc_circulating_supply,trading_pair
timestamp,,,,
2025-10-02 10:00:00,91685.618,10893897738.8281765,19928203,BTC-USDT
2025-10-02 11:00:00,91697.175,10873024352.51871109,19928203,BTC-USDT
2025-10-02 12:00:00,91853.679,10904790361.78449821,19928203,BTC-USDT
2025-10-02 13:00:00,92256.13,11011798337.58810425,19928203,BTC-USDT
2025-10-02 14:00:00,93393.434,11166908638.94875908,19928203,BTC-USDT


## Create Visualization

In [20]:
# Create subplots with secondary y-axis
fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.03,
    row_heights=[0.5, 0.3],
    specs=[
        [{"secondary_y": True}],
        [{"secondary_y": True}]
    ]
)

# Get candles dataframe
candles_df = candles.data

# 1. Add candlestick chart
fig.add_trace(
    candles.candles_trace(),
    row=1, col=1,
    secondary_y=False
)

# 2. Add open interest data if available
if not oi_data.empty:
    # Plot open interest value
    fig.add_trace(
        go.Scatter(
            x=oi_data.index,
            y=oi_data['sum_open_interest'],
            mode='lines+markers',
            name='Open Interest',
            line=dict(color='orange', width=2),
            marker=dict(size=4),
            showlegend=True
        ),
        row=2, col=1,
        secondary_y=False
    )
    
    # Plot open interest value in USD
    if 'sum_open_interest_value' in oi_data.columns:
        fig.add_trace(
            go.Scatter(
                x=oi_data.index,
                y=oi_data['sum_open_interest_value'],
                mode='lines',
                name='OI Value (USD)',
                line=dict(color='green', width=1, dash='dash'),
                showlegend=True
            ),
            row=2, col=1,
            secondary_y=True
        )
    
    # Add OI overlay on price chart (normalized)
    if len(oi_data) > 0:
        # Normalize OI to price range for overlay
        price_range = candles_df['high'].max() - candles_df['low'].min()
        price_mean = candles_df['close'].mean()
        oi_normalized = (oi_data['sum_open_interest'] - oi_data['sum_open_interest'].min()) / \
                       (oi_data['sum_open_interest'].max() - oi_data['sum_open_interest'].min() + 1e-8)
        oi_scaled = oi_normalized * price_range * 0.2 + candles_df['low'].min()
        
        fig.add_trace(
            go.Scatter(
                x=oi_data.index,
                y=oi_scaled,
                mode='lines',
                name='OI (scaled)',
                line=dict(color='purple', width=1),
                opacity=0.3,
                showlegend=True
            ),
            row=1, col=1,
            secondary_y=True
        )

# Update layout
fig.update_layout(
    title=f"{candles.trading_pair} - Price, Volume & Open Interest Analysis",
    xaxis_title="Time",
    height=900,
    hovermode='x unified',
    showlegend=True,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)

# Update y-axes labels
fig.update_yaxes(title_text="Price", row=1, col=1, secondary_y=False)
fig.update_yaxes(title_text="OI Scaled", row=1, col=1, secondary_y=True)
fig.update_yaxes(title_text="Open Interest", row=2, col=1, secondary_y=False)
fig.update_yaxes(title_text="OI Value (USD)", row=2, col=1, secondary_y=True)

# Update x-axis and remove rangeslider
fig.update_xaxes(title_text="Date", row=1, col=1, rangeslider_visible=False)

# Show the plot
fig.show()